In [36]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

In [37]:
using Gmsh
using Gridap
using GridapGmsh


gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 0);
ms_max = 1;
ms_min = 0.1;
MSFC = 3;
EL_ORDER = 1
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", MSFC);
# gmsh.option.setNumber("Mesh.MeshSizeMax", diam/2);
# gmsh.option.setNumber("Mesh.MeshSizeMin", diam/5);
gmsh.option.setNumber("Mesh.MaxNumThreads3D", 8);
gmsh.option.setNumber("Mesh.ElementOrder", EL_ORDER);
# gmsh.option.setNumber("Mesh.Algorithm3D", 9);
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2);

gmsh.model.add("L_2_1")

len = 1;
height = 0.2;
width = 0.2


Lc1 = 0.05;

factory = gmsh.model.geo;
factory.addPoint(  0,      0,  0, Lc1, 1);
factory.addPoint(len,      0,  0, Lc1, 2);
factory.addPoint(len, height,  0, Lc1, 3);
factory.addPoint(  0, height,  0, Lc1, 4);


factory.addLine(1, 2, 1);
factory.addLine(2, 3, 2);
factory.addLine(3, 4, 3);
factory.addLine(4, 1, 4);



factory.addPhysicalGroup(1, [1, 3], -1, "free");
factory.addPhysicalGroup(1, [2], -1, "loaded");
factory.addPhysicalGroup(1, [4], -1, "fixed");

factory.synchronize();

factory.addCurveLoop([1,2,3,4], 5);



factory.addPlaneSurface([5], 6)


factory.synchronize()

factory.addPhysicalGroup(2, [6], -1, "beam");

factory.synchronize()

gmsh.model.mesh.generate(2)


name = "beam.msh" |> tcf;
print(name)
gmsh.write(name)

gmsh.finalize()


model = GmshDiscreteModel(name);

vtk_file = "model" |> tcf;
writevtk(model, vtk_file);

c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\beam.mshInfo    : Reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\beam.msh'...
Info    : 9 entities
Info    : 128 nodes
Info    : 254 elements
Info    : Done reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\beam.msh'


In [38]:
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64}, EL_ORDER)

V = TestFESpace(model,reffe;
  conformity=:H1,
  dirichlet_tags=["fixed"],
  dirichlet_masks=[(true,true)]
  )

UnconstrainedFESpace()

In [39]:
g(x) = VectorValue(0.0, 0.0)

g (generic function with 1 method)

In [40]:
U = TrialFESpace(V,[g])

TrialFESpace()

In [45]:
const E = 2.1e11
const ν = 0.3
const ρ = 7800
const gravity = 9.8

function σ(eps)
    λ = E*ν/(1+ν)/(1-2*ν)
    μ = E/2/(1+ν)
    return λ*tr(eps)*one(eps) + 2*μ*(eps)
end

σ (generic function with 1 method)

In [46]:
degree = 2*EL_ORDER
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [51]:
f = VectorValue(0.0, - gravity * ρ)
a(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)))*dΩ
l(v) = ∫(dot(f, v))*dΩ

l (generic function with 1 method)

In [52]:
op = AffineFEOperator(a,l,U,V)
uh = solve(op)

SingleFieldFEFunction():
 num_cells: 206
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 522409693027647030

In [53]:
res_file = "results" |> tcf;
writevtk(Ω, res_file,cellfields=["uh"=>uh,"epsi"=>ε(uh),"sigma"=>σ∘ε(uh)])

(["c:\\Users\\kutsj\\OneDrive\\Education\\Programming\\Julia\\CCM_tut\\lesson_2\\results.vtu"],)